In [2]:
import FlockModel
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time

Viscek:
Stated briefly as rules, and in $\textbf{order of decreasing precedence}$, the behaviors that lead to simulated flocking are:
1. Collision Avoidance: avoid collisions with nearby
flockmates
2. Velocity Matching: attempt to match velocity with nearby
flockmates
3. Flock Centering: attempt to stay close to nearby flockmates 

# Show that increasing centralization, vc and angle increases convergence to order

In [5]:
importlib.reload(FlockModel)

align = [3]
centering = [1]
repulsion = [2]
pred_repulsion = [5]
pred_phenotype = [[1,0.25,1,1]]

for a in align:
    for c in centering:
        for r1 in repulsion:
            for r2 in pred_repulsion:
                for pred_pheno in pred_phenotype:
                    phenotype = [a,c,r1,r2,0.5]
                    title = str(phenotype) + str(pred_pheno)
                    model = FlockModel.Model(maxtime = 25, pred_phenotype = pred_pheno, dt = 0.25, noise = 0.05, L = 25, num_prey = 500, num_pred = 0, prey_phenotype = phenotype)
                    model.run()
                    model.animate(title = title)
                    flock_size, exclusion_radius = model.clustering(n = 500, save = True,title = title + " Clustering Plot")
                    model.order_plot(save = True)
                    model.vel_plot(save = True)



<Figure size 2400x1600 with 0 Axes>

<Figure size 2400x1600 with 0 Axes>

# Milling

In [ ]:
importlib.reload(FlockModel)

model = FlockModel.Model(dt = 0.25, maxtime = 10, noise = 0.1, phenotype = [0,1,0], density=1, angle=2*np.pi, L=2, predators=0, bc="soft", exc_r=0.05)
st = time.time()
model.run()
et = time.time()
print("Model elapsed time: ", et-st, " seconds")
st = time.time()
model.animate(name="correlation_test")
et = time.time()
print("Animation elapsed time: ", et-st, " seconds")

In [ ]:
importlib.reload(FlockModel)

model = FlockModel.Model(dt = 0.1, maxtime = 40, noise = 0.1, phenotype = [0,1,0], density=8, angle=2*np.pi, L=4, predators=0, bc="soft", exc_r=0.05)
st = time.time()
model.run()
et = time.time()
print("Model elapsed time: ", et-st, " seconds")
st = time.time()
model.animate(name="correlation_test")
et = time.time()
print("Animation elapsed time: ", et-st, " seconds")

# Clustering test

In [ ]:
importlib.reload(FlockModel)

model = FlockModel.Model(dt = 0.25, maxtime = 30, noise = 0.1, phenotype = [0,1,0], density=3, L=30, predators=0)
st = time.time()
model.run()
et = time.time()
print("Model elapsed time: ", et-st, " seconds")
# model.animate(name="correlation_test")

In [ ]:
importlib.reload(FlockModel)

model = FlockModel.Model(dt = 0.25, maxtime = 30, noise = 0.1, phenotype = [0,1,0], density=3, L=30, predators=0, bc="soft")
st = time.time()
model.run()
et = time.time()
print("Model elapsed time: ", et-st, " seconds")
model.animate(name="correlation_test")

In [ ]:
model.animate_cluster_plot()

# Connected correlation function test

In [ ]:
importlib.reload(FlockModel)

model = FlockModel.Model(dt = 0.25, maxtime = 20, noise = 0.1, phenotype = [0,1,0], density=1, L=10, predators=2, bc="soft")
st = time.time()
model.run()
et = time.time()
print("Model elapsed time: ", et-st, " seconds")
model.animate(name="correlation_test")
model.order_plot()
# model.sus_plot()

In [ ]:
model.quiver_plot(i=3)
model.quiver_plot(i=9)
model.vel_fluc_plot(i=3)
model.vel_fluc_plot(i=9)
model.corr_plot(i=3)
model.corr_plot(i=9)